The following notebook gives a SageMath implementation to compute an algebraic equation satisfied by the **main diagonal** 
$$ d(t) = \sum_{n \geq 0} f_{n,n}t^n $$
of a bivariate rational function $F(x,y)$ with power series expansion
$$ F(x,y) = \sum_{i,j \geq 0} f_{i,j} x^i y^j.$$

The method comes from the paper *Algebraic diagonals and walks: algorithms, bounds, complexity* by Bostan, Dumont, and Salvy (Journal of Symbolic Computation, Volume 83, Pages 68-92, 2017). 

This implementation was created by Stephen Melczer and Peiran Tao (both at the University of Waterloo). It is still undergoing full testing -- feel free to send any comments or bug reports to the authors.

In [1]:
def AlgebraicResidues(P,Q,S):
    r"""Compute annihilating polynomial of the residues of a rational function at the zeroes of another polynomial.
    Taken from Algorithm 1 of Bostan, Dumont, and Salvy (2017).

    INPUT:

    * ``P``      -- Polynomial in K[y].
    * ``Q``      -- Polynomial in K[y] with non-zero constant.
    * ``S``      -- Polynomial in K[y] that is a divisor of Q such that S and Q/S are coprime (S can be Q).

    OUTPUT:

    A polynomial R(z) in K[z] that annihilates the residues of P/Q at the roots of S.
    """
    # Build needed Sage objects
    R = P.parent()                                 # R = K[y]
    y = R.gen()
    B.<t> = PowerSeriesRing(FractionField(R), 't') # B = K(y)[[t]]

    # Compute square-free decomposition of S
    decomp = list(S.squarefree_decomposition())
    m = len(decomp)
    Result = [1] * m

    # Iterate over factors in square-free decomposition
    for i in range(m):
        # decomp[i] = (Q_{i+1}, d_{i+1})
        Qi = decomp[i][0]
        di = decomp[i][1]

        if Qi.degree() == 0:
            R[i] = 1                                # Ri = 1
        else:
            Ui = Q // Qi^di                         # Ui = Q/Q_i^i
            Vi = (Qi(y+t) - Qi(y)).quo_rem(t)[0]    # Vi = (Qi(y+t)-Qi(y))/t

            # Take coefficient Si = [t^(di-1)] P(y+t) / (Ui(y+t) * Vi^di) 
            # by expanding the rational function as a power series to O(t^di)
            Di = Ui(y+t) * Vi^di
            Yi = P(y+t) / (Di + O(t^di))
            Si = Yi[di-1]

            # Extract numerator and denominator of Si, and make sure they are coprime
            Ai = Si.numerator()
            Bi = Si.denominator()
            G = Ai.gcd(Bi)
            Ai = Ai // G
            Bi = Bi // G 

            # Create polynomial rings to perform resultant computation
            Lz.<z> = PolynomialRing(R.base_ring(), 'z') # Lz = K[z]
            Ly.<y> = PolynomialRing(Lz, 'y')            # Ly = K[z][y]

            # Compute desired resultant for the term in the square-free decomposition
            H = Ly(Ai) - z * Ly(Bi)
            Result[i] = H.resultant(Ly(Qi))  # Resultant taken over y, so result is in K[z]

    return (prod(Result[i] for i in range(m))).monic() # Other formulas require our output to be monic

In [2]:
def PureComposedSum(P, c, var):
    r"""Compute annihilating polynomial for all sums of c roots of a polynomial.
    Taken from Algorithm 2 of Bostan, Dumont, and Salvy (2017).

    INPUT:

    * ``P``      -- A univariate or multivariate polynomial WITHOUT y and z among its variables
    * ``c``      -- An integer between 1 and deg(P) (inclusive)
    * ``var``    -- A string or variable specifying a variable that appears in P

    OUTPUT:

    A polynomial (Σ_cP)(y) in K[y] whose roots are the sums of all c-subsets of the roots of P.
    """

    # TODO: Add tests that var is a variable of P and that z and y do not appear in P
    # TODO: Fix overloading of Y and y in univariate and multivariate rings

    # Convert P to a polynomial in var 
    # K is the polynomial ring in the remaining variables
    P_var = P.polynomial(P.parent()(var))
    K = P_var.base_ring()
    d = P_var.degree()
    D = binomial(d,c)

    # Create rings needed for our computations
    R1 = PolynomialRing(K, names=(var, 'z'))
    R2.<Y> = K[['Y']]
    y, z = R1.gens()

    # Cast P into K[[Y]]
    P_Y = P_var(Y)
    P_Y_der = P_Y.derivative()

    # Compute reciprocal polynomials of P and P' (polynomials with reversed coefficient sequences)
    recP = Y^d * P_Y(1/Y) 
    recP_der = Y^(d-1) * P_Y_der(1/Y)

    # Introduce bivariate power series ring
    R3.<Y,Z> = K[['Y,Z']]

    # Compute terms in generating function for Newton Sum of P and take Hadamard product with exp(Y)
    # Note: for now S lies in K[[Y,Z]] even though it depends only on Y
    NP = recP_der/(recP + O(Y^(D+1)))
    S = sum((NP[n] / factorial(n)) * Y^n for n in range(D+1))

    # Compute intermediate sum, take its exponential, and extract desired term
    Sum = sum((-1)^(n-1) * (S(n*Y,0) / n) * Z^n for n in range(1,c+1))
    F = (Sum + O(Y^(c+D+1))).exp()
    Fc = sum(C * y^e[0] for e, C in F.polynomial().dict().items() if e[0] < D+1 and e[1] == c)
    
    # Compute the Newton Sum of Σ_cP and cast into K[[Y]]
    NSumP = sum(Fc[n,0] * factorial(n) * Y^n for n in range(D+1))
    NSumP_Y = R2(NSumP)

    # Recover Σ_cP by integrating and taking an exponential
    integrand = (D - NSumP_Y) / R2(Y)
    IntegralEXP = (integrand.integral()+O(R2(Y)^(D+1))).exp()
    U = IntegralEXP.truncate(D+1)(y)
    recU = y^(U.degree()) * U(1/y,0)

    R4.<y> = PolynomialRing(K, 'y')
    return R4(recU(y,0))

In [3]:
def AlgebraicDiagonal(f, params=[], output_vars=None):
    r"""Compute annihilating polynomial for the (main) diagonal of a bivariate rational function.
    Taken from Algorithm 3 of Bostan, Dumont, and Salvy (2017).

    INPUT:

    * ``f``            -- A symbolic rational function with two variables (can have additional symbolic parameters)
    * ``params``       -- (optional) List of variables in f that should be considered parameters
    * ``output_vars``  -- (optional) Variables for the resulting polynomial (if None then output is polynomial in t and y)

    OUTPUT:

    A polynomial P(t,y) in K[t][y] such that the diagonal d(t) of f satisfies P(t,d(t)) = 0,
    where output_vars = (t,y). Note, in particular, that the dependent variable for the diagonal
    is the first element of output_vars.
    """

    # TODO: Verify local variable names don't clash with input variables
    # TODO: Take more care with variable names in final section of code
    # TODO: Fix case with parameters (need to use proof.WithProof('polynomial', False) when factoring) 
    
    # Extract variables and define ring structure
    f_variables = [v for v in f.variables() if v not in params]

    if len(params) == 0:
        K = QQ
    else:
        raise NotImplementedError("The case with parameters is still in progress")
        
    R = PolynomialRing(K, names = f_variables)

    # Verify f has two (non-parameter) variables
    if len(f_variables) != 2:
        raise ValueError("Input f needs to have exactly 2 non-parameter variables")
        
    local_x, local_y = R.gens()

    # Verify f is a bivariate rational function (possibly with parameters)
    try:
        A, B = R(f.numerator()), R(f.denominator())
    except:
        raise ValueError("Input f not a rational function in all variables and parameters")

    # Compute needed quantities
    dA = max(i - j for (i,j) in A.dict())
    dB = max(i - j for (i,j) in B.dict())
    a = dB - dA - 1

    # Compute number of branches that go to 0 as the dependent variable goes to 0
    pol = R(local_y^dB * B(local_x/local_y, local_y))
    pol_star = prod([p for (p,_) in pol.factor()])
    pol_star_lowest = pol_star.polynomial(local_x)[pol_star.polynomial(local_x).valuation()]
    c = pol_star_lowest.polynomial(local_y).valuation()

    # Define quantities we use to compute residues
    Kt.<t> = FractionField(K['t'])
    Kt_Y.<Y> = Kt['Y']
    P = Y^dA * A(t/Y, Y)
    Q = Y^dB * B(t/Y, Y)
    
    Pt = Kt_Y(P)
    Qt = Kt_Y(Q)

    # Compute residues -- note formula in Bostan, Dumont, and Salvy (2017) has typo when a < 0
    if a < 0:
        R = AlgebraicResidues(Pt, Y^(-a) * Qt, Qt)
    else:
        R = AlgebraicResidues(Y^a * Pt, Qt, Qt)
    
    T.<Z> = (parent(R).base_ring())['Z']
    R = R(Z)
    
    Phi = (PureComposedSum(R,c,'Z')).numerator()
    
    if a < 0:
        r = AlgebraicResidues(Pt, Y^(-a)*Qt, Y^(-a))
        r = parent(Phi)(r)
    
    # Determine desired output variables
    y = parent(Phi)(local_y)    
    if output_vars == None:
        output_t, output_y  = (t, y)
    elif len(output_vars) == 2:
        output_t, output_y  = output_vars
    else:
        raise ValueError("output_vars needs to specify exactly 2 variables")

    Output_Ring = QQ[output_t][output_y]

    # Convert to the desired output variables and return result
    if a < 0:
        return Output_Ring(Phi(r).numerator().subs(t=output_t, y=output_y))
    else:
        return Output_Ring(Phi.subs(t=output_t, y=output_y))

We end with some tests of the implementation.

In [4]:
var('x y a b')

(x, y, a, b)

In [5]:
# Example 1: 
# The diagonal of F = 1/(1-x-y) is (1-4t)^(-1/2)
# This returns (t-1/4)y^2 + 1/4
AlgebraicDiagonal(1/(1-x-y))

(t - 1/4)*y^2 + 1/4

In [6]:
# Example 2: Example 6.3.8 from Stanley's Enumerative Combinatorics Vol II
# The diagonal of F = 1/(1-x-y-xy) is (1-6t+t^2)^(-1/2)
# This gives (t^2-6t+1)y^2 - 1
AlgebraicDiagonal(1/(1-x-y-x*y))

(t^2 - 6*t + 1)*y^2 - 1

In [7]:
# Example 3: Example 6.3.9 from Stanley's Enumerative Combinatorics Vol II
# The diagonal of F = (1-x)(1-y)/(1-2x-2y+2xy) is 1/2 * (1+(1-12t+4t^2)^(-1/2))
# Expect (t^2 - 3*t + 1/4)*y^2 + (-t^2 + 3*t - 1/4)*y + 1/4*t^2 - 3/4*t
A = (1-x)*(1-y)
B = 1-2*x-2*y+2*x*y
AlgebraicDiagonal(A/B)

(t^2 - 3*t + 1/4)*y^2 + (-t^2 + 3*t - 1/4)*y + 1/4*t^2 - 3/4*t

In [8]:
# Example 4:
# The diagonal of F = (2*y^3*x + 3*y^2*x + 2*y*x - y + x - 1)/(y^2*x + 2*y*x + x - 1)
# is the Catalan generaing function. This gives ty^2 - y + 1.
A = 2*y^3*x + 3*y^2*x + 2*y*x - y + x - 1
B = y^2*x + 2*y*x + x - 1
AlgebraicDiagonal(A/B)

t*y^2 - y + 1

In [9]:
# Example 5: Bicoloured supertrees
# The diagonal of this function is a root of y^4 - 2*y^3 + (2*t + 1)*y^2 - 2*t*y + 4*t^3
A = 2*x^2*y*(2*x^5*y^2 - 3*x^3*y + x + 2*x^2*y - 1)
B = x^5*y^2 + 2*x^2*y - 2*x^3*y + 4*y + x - 2
AlgebraicDiagonal(A/B) 

y^4 - 2*y^3 + (2*t + 1)*y^2 - 2*t*y + 4*t^3

In [10]:
AlgebraicDiagonal(A/B, output_vars=[a,b]) 

b^4 - 2*b^3 + (2*a + 1)*b^2 - 2*a*b + 4*a^3